# Experimento para analizar la convergencia de los procesos de optmización usando diferentes optmizadores y evluando distintos escenarios de configuración molecular de la molecula de Hidrógeno.

In [1]:
# @title
!pip install openfermion pyscf ipywidgets matplotlib dimod openfermionpyscf

  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.1 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13820 sha256=b04fdd4d6fa0e560f18b2babc773879f7f69d07f7d738819b7d5bfcf9370be9f
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy


In [5]:
# @title
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import dimod
from openfermionpyscf import run_pyscf
from openfermion import MolecularData, get_fermion_operator, jordan_wigner
from pyscf import gto, scf

# Función para visualizar la molécula en 3D con Matplotlib
def plot_molecule_3d(geometry):
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Extraer las posiciones de los átomos
    atom_positions = np.array([atom[1] for atom in geometry])
    ax.scatter(atom_positions[:, 0], atom_positions[:, 1], atom_positions[:, 2], s=100, c='r', marker='o')

    # Etiquetas de los átomos
    for i, atom in enumerate(geometry):
        ax.text(atom[1][0], atom[1][1], atom[1][2], atom[0], fontsize=12, color='black')

    # Configurar los ejes
    ax.set_xlabel('X (Å)')
    ax.set_ylabel('Y (Å)')
    ax.set_zlabel('Z (Å)')
    ax.set_title('Molécula en 3D')

    plt.show()

# Función para mapear Hamiltoniano_qubit a Hamiltoniano_qubo
def qubit_hamiltonian_to_qubo(qubit_hamiltonian):
    qubo = {}
    offset = 0.0

    for term, coeff in qubit_hamiltonian.terms.items():
        qubits = []
        is_qubo = True

        # Iterar sobre los operadores de Pauli y asegurarse de que solo haya términos de Z o identidad
        for qubit, pauli in term:
            if pauli == 'Z':  # Solo los términos Z pueden mapearse directamente a QUBO
                qubits.append(qubit)
            else:
                is_qubo = False
                break

        if not is_qubo:
            continue

        # Asignar el término al diccionario QUBO
        if len(qubits) == 0:  # Término constante
            offset += coeff.real
        elif len(qubits) == 1:  # Término lineal (un solo qubit)
            qubit = qubits[0]
            qubo[(qubit, qubit)] = qubo.get((qubit, qubit), 0) + coeff.real
        elif len(qubits) == 2:  # Término cuadrático (interacción entre dos qubits)
            qubit1, qubit2 = qubits
            qubo[(qubit1, qubit2)] = qubo.get((qubit1, qubit2), 0) + coeff.real

    return qubo, offset

# Función principal que optimiza la longitud de enlace y calcula los resultados
def optimize_bond_length(distances, basis_list, optimizer, num_reads, progress_bar):
    results = {}
    total_steps = len(basis_list) * len(distances)
    current_step = 0

    for basis in basis_list:
        mean_energies = []

        for dist in distances:
            # Definir la geometría de la molécula de H2 con la distancia de enlace actual
            geometry = [('H', (0, 0, 0)), ('H', (0, 0, dist))]

            # Definir la molécula en PySCF con la base actual
            mol = gto.Mole()
            mol.build(
                atom='H 0 0 0; H 0 0 {}'.format(dist),
                basis=basis,
                unit='angstrom',
                verbose=0  # Deshabilitar la salida
            )

            # Calcular el Hartree-Fock con PySCF
            mf = scf.RHF(mol)
            mf.kernel()

            # Almacenar los datos moleculares en un objeto MolecularData de OpenFermion
            molecular_data = MolecularData(
                geometry=geometry,
                basis=basis,
                multiplicity=1,
                charge=0
            )

            # Ejecutar PySCF con OpenFermion para obtener los integrales
            molecular_data = run_pyscf(molecular_data, run_scf=True, run_fci=True)

            # Obtener el Hamiltoniano molecular en formato de fermiones
            hamiltonian = molecular_data.get_molecular_hamiltonian()

            # Convertir a qubits usando la transformación de Jordan-Wigner
            qubit_hamiltonian = jordan_wigner(get_fermion_operator(hamiltonian))

            # Convertir a QUBO
            qubo, offset = qubit_hamiltonian_to_qubo(qubit_hamiltonian)

            # Crear un BinaryQuadraticModel a partir de la matriz QUBO
            bqm = dimod.BinaryQuadraticModel.from_qubo(qubo)

            # Usar el optimizador seleccionado
            if optimizer == "Annealing Simulado":
                sampler = dimod.SimulatedAnnealingSampler()
            elif optimizer == "Annealing Cuántico":
                sampler = dimod.ExactSolver()  # Cambiar a un sampler cuántico si es necesario
            else:
                sampler = dimod.ExactSolver()

            # Realizar el muestreo
            solution = sampler.sample(bqm, num_reads=num_reads)

            # Extraer la energía media
            energies = [sample.energy for sample in solution.data()]
            mean_energy = np.mean(energies)
            mean_energies.append(mean_energy)

            # Actualizar la barra de progreso
            current_step += 1
            progress_bar.value = (current_step / total_steps) * 100

        # Guardar los resultados de energía para la base actual
        results[basis] = mean_energies

    # Completar la barra de progreso
    progress_bar.value = 100

    return results

# Función para actualizar las gráficas dinámicas
def update_plot(bases, optimizer, distance_min, distance_max, num_points, num_reads):
    # Generar el rango de distancias
    distances = np.linspace(distance_min, distance_max, num_points)

    # Limpiar el contenido actual del gráfico
    energy_box.clear_output(wait=True)

    # Crear y mostrar barra de progreso
    with progress_bar_box:
        progress_bar.value = 0  # Reiniciar la barra de progreso

    # Llamar a la función de optimización
    results = optimize_bond_length(distances, bases, optimizer, num_reads, progress_bar)

    # Graficar los resultados de energía vs longitud de enlace
    with energy_box:
        fig, ax = plt.subplots(figsize=(8, 6))
        for basis, energies in results.items():
            ax.plot(distances, energies, marker='o', label=f'Base: {basis}')

        ax.set_xlabel('Distancia de enlace (Å)')
        ax.set_ylabel('Energía media (Hartree)')
        ax.set_title(f'Energía vs Distancia de enlace para {optimizer}')
        ax.grid(True)
        ax.legend()
        plt.show()

# Crear widgets
bases_select = widgets.SelectMultiple(
    options=['sto-3g', '6-31g', 'cc-pvdz', 'cc-pvtz'],
    value=['sto-3g'],
    description='Bases:'
)

optimizer_select = widgets.RadioButtons(
    options=['Annealing Simulado', 'Annealing Cuántico', 'Exacto'],
    value='Annealing Simulado',
    description='Optimizador:'
)

distance_min_slider = widgets.FloatSlider(
    value=0.5, min=0.3, max=3.0, step=0.1,
    description='Dist Min (Å):',
    continuous_update=False
)

distance_max_slider = widgets.FloatSlider(
    value=2.0, min=0.3, max=3.0, step=0.1,
    description='Dist Max (Å):',
    continuous_update=False
)

num_points_slider = widgets.IntSlider(
    value=10, min=5, max=100, step=5,
    description='Num Puntos:',
    continuous_update=False
)

num_reads_slider = widgets.IntSlider(
    value=100, min=100, max=5000, step=100,
    description='Num Lecturas:',
    continuous_update=False
)

# Crear el botón de actualización
update_button = widgets.Button(description="Actualizar Gráfica")

# Crear la barra de progreso global
progress_bar = widgets.FloatProgress(
    value=0,
    min=0,
    max=100,
    description='Progreso:',
    bar_style='info'
)
progress_bar_box = widgets.Output()
with progress_bar_box:
    display(progress_bar)

# Definir la acción del botón
def on_update_button_clicked(b):
    update_plot(
        bases=bases_select.value,
        optimizer=optimizer_select.value,
        distance_min=distance_min_slider.value,
        distance_max=distance_max_slider.value,
        num_points=num_points_slider.value,
        num_reads=num_reads_slider.value
    )

update_button.on_click(on_update_button_clicked)

# Mostrar molécula en 3D con Matplotlib
mol_geometry = [('H', (0, 0, 0)), ('H', (0, 0, 0.7414))]
#plot_molecule_3d(mol_geometry)

# Organizar las cajas superiores (gráficas)
molecule_box = widgets.Output()
energy_box = widgets.Output()

with molecule_box:
    plot_molecule_3d(mol_geometry)

# Organizar las cajas inferiores (widgets de personalización)
controls_box = widgets.VBox([
    bases_select, optimizer_select, distance_min_slider,
    distance_max_slider, num_points_slider, num_reads_slider, update_button
])

# Organizar el layout general
top_box = widgets.HBox([molecule_box, energy_box])
main_layout = widgets.VBox([top_box, controls_box, progress_bar_box])

# Mostrar la interfaz completa
display(main_layout)
